### Step 1: Creating Token

In [32]:
with open(r"D:\Data_Science_Study\Course_Pracitse_Code\GenerativeAi\LLM_From_Scratch\Building-LLMs-from-Scratch\0. Data\the-verdict.txt" , "r" , encoding = "utf-8") as f:
    raw_text = f.read()


print("Total Number of Character:", len(raw_text))
print(raw_text[:99])

Total Number of Character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### Our Aim is to tokenize this 20-479 chracter short story into individual words and special character that we can trun into embedding for LLM training

### Split text to Token

### Test Example to see How we can convert to each token

In [33]:
import re 

text = "Hello, world. this , is a test"

result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'this', ' ', ',', ' ', 'is', ' ', 'a', ' ', 'test']


### Split on comma[,] and period[.]

In [34]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'this', ' ', '', ',', '', ' ', 'is', ' ', 'a', ' ', 'test']


### Still White space is there in token Let's remove those

In [35]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'this', ',', 'is', 'a', 'test']


### Removing Spces or Not

### That is Depend on Dataset : Example Python code dataset where space have meaning.... Removing whitespaees reduce the memory and computing requirement.

In [36]:
result = re.split(r'([,.!?:;"()\']|--|\s)', text)
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'this', ',', 'is', 'a', 'test']


In [37]:
text = "Hello, world. this , -- is a test"

result = re.split(r'([,.!?:;"()\']|--|\s)', text)
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'this', ',', '--', 'is', 'a', 'test']


### Applying to our dataset

In [38]:
preprocessed = re.split(r'([,._!?:;"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]

print(len(preprocessed))
print(preprocessed[:35])

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory']


### Step :2 Creating Token ID

In [39]:
all_words = sorted(set(preprocessed))

vocab_size = len(all_words)

print(vocab_size)

1130


In [40]:
vocab = {token:integer for integer, token in enumerate(all_words)}

In [41]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


### Creating simple Tokennizer Class

In [42]:
class SimpleTokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s ,i in vocab.items()}
    

    def encode(self,text):
        preprocessed = re.split(r'([,._!?:;"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]

        ids = [self.str_to_int[s] for s in preprocessed]

        return ids
    

    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        # replace spaces for pamctuations
        text  = re.sub(r'\s+([,.?!"()\'])', r'\1',text)

        return text

In [43]:
tokenizer = SimpleTokenizer(vocab)

text = """It's the last he painted, you know,
            Mrs, Gisburn said with pardonable pride"""


ids = tokenizer.encode(text)

print(ids)


[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 67, 5, 38, 851, 1108, 754, 793]


In [44]:
print(tokenizer.decode(ids))

It' s the last he painted, you know, Mrs, Gisburn said with pardonable pride


### What if word is not in vocab???

In [45]:

text = """"It's the last he painted, you know,
            Mrs, Gisburn said with pardonable pride. Hello do you like in tea??"""


ids = tokenizer.encode(text)

print(ids)


KeyError: 'Hello'

### Hello is not there so it gave error... 
### Adding Special Context Tokens

In [46]:
all_tokens = sorted(list(set(preprocessed)))

all_tokens .extend(["<|endoftext|>","<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

print(len(vocab.items()))

1132


In [47]:
for i,item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [ ]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s ,i in vocab.items()}
    

    def encode(self,text):
        preprocessed = re.split(r'([,._!?:;"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]

        return ids
    

    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        # replace spaces for pamctuations
        text  = re.sub(r'\s+([,.?!"()\'])', r'\1',text)
        
        return text

In [50]:
tokenizer = SimpleTokenizerV2(vocab)

text1  = "hello, do you like tea?"
text2 = "In the sunlit terraces of the palace"

text = "<|endoftext|> ".join((text1,text2))

print(text)

hello, do you like tea?<|endoftext|> In the sunlit terraces of the palace


In [51]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131]

In [52]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>'

### Some more token can be added :
### <BOS> : Begining of text
### <EOS> : End of text
### [PAD] : Padding ( batch may have varriying length so to see batch have same length of text size we can agument [PAD] etc)

### Byte Pair Encoding

In [53]:
import tiktoken

In [64]:
tokenizer  = tiktoken.get_encoding('gpt2')

In [65]:
text = (
    "Hello , do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunkownPlace."
)

In [66]:
integers = tokenizer.encode(text , allowed_special={"<|endoftext|>"})
print(integers)

[15496, 837, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 2954, 593, 27271, 13]


In [67]:
strings = tokenizer.decode(integers)
print(strings)

Hello , do you like tea? <|endoftext|> In the sunlit terracesof someunkownPlace.
